In [ ]:
# Install runtime deps
!pip install -q fastapi uvicorn nest_asyncio pyngrok

#  mount Google Drive to load checkpoints from /content/drive/MyDrive
from google.colab import drive
drive.mount('/content/drive')


CKPT_CNN  = "/content/drive/MyDrive/cnn_model.pth"
CKPT_LSTM = "/content/drive/MyDrive/lstm_model.pth"


Mounted at /content/drive


In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

DEVICE = "cpu"

# ---- Models  ----
class EEG_CNN(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.conv1 = nn.Conv1d(14, 32, kernel_size=3)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3)
        self.pool  = nn.AdaptiveAvgPool1d(1)
        self.fc1   = nn.Linear(64, 32)
        self.fc2   = nn.Linear(32, num_classes)
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x).squeeze(-1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

class EEG_LSTM(nn.Module):
    def __init__(self, input_size=14, hidden_size=64, num_layers=2, num_classes=2):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers  = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1  = nn.Linear(hidden_size, 32)
        self.fc2  = nn.Linear(32, num_classes)
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, device=x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, device=x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = F.relu(self.fc1(out[:, -1, :]))
        return self.fc2(out)

# ---- Normalization (per sample, per channel; same as eval) ----
def normalize_per_sample(sample_14_T: np.ndarray) -> np.ndarray:
    mean = sample_14_T.mean(axis=1, keepdims=True)
    std  = sample_14_T.std(axis=1, keepdims=True)
    std[std == 0] = 1
    return (sample_14_T - mean) / std

# ---- Model loading ----
def load_cnn(ckpt_path: str) -> nn.Module:
    m = EEG_CNN().to(DEVICE)
    m.load_state_dict(torch.load(ckpt_path, map_location=DEVICE))
    m.eval()
    return m

def load_lstm(ckpt_path: str) -> nn.Module:
    m = EEG_LSTM().to(DEVICE)
    m.load_state_dict(torch.load(ckpt_path, map_location=DEVICE))
    m.eval()
    return m


In [ ]:
# ---------------- FastAPI ----------------
from fastapi import FastAPI, UploadFile, File

app = FastAPI()

@app.get("/")
def root():
    return {"status": "ok", "message": "EEG API running. Visit /docs for Swagger UI."}

@app.post("/predict_cnn")
async def predict_cnn(file: UploadFile = File(...)):
    data = np.load(file.file)   # must be (14, T)
    result = predict_model(cnn_model, data, "CNN")
    return {"model": "CNN", "prediction": result}

@app.post("/predict_lstm")
async def predict_lstm(file: UploadFile = File(...)):
    data = np.load(file.file)   # must be (14, T)
    result = predict_model(lstm_model, data, "LSTM")
    return {"model": "LSTM", "prediction": result}


In [ ]:
import os
os.environ["NGROK_AUTH_TOKEN"] = "330PuAWTfTyPYHXGy5iVtC1ajhx_2URNNdK974QAAXTpwG62C"

In [ ]:
import os
import threading
import nest_asyncio
import uvicorn
from pyngrok import ngrok



token = os.getenv("NGROK_AUTH_TOKEN")
if token:
    ngrok.set_auth_token(token)


nest_asyncio.apply()

# Start public tunnel to port 8000
public_url = ngrok.connect(8000, "http")
print(" Public URL:", public_url)

# Run FastAPI app in a background thread
def run_app():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

thread = threading.Thread(target=run_app, daemon=True)
thread.start()

print(" Server started. Open the Public URL above, then add /docs")


 Public URL: NgrokTunnel: "https://429d7e864b3f.ngrok-free.app" -> "http://localhost:8000"
 Server started. Open the Public URL above, then add /docs
